In [ ]:
#Imports
import torch
from torchtext.legacy import data
import pandas as pd
import re
import string
import random
import os 
import torch.nn as nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


In [ ]:
#dataset upload
from google.colab import files
uploaded = files.upload()

Hindi Dataset

In [ ]:
# Read Hinid dataset and some preprocessing
# Read Hindi Dataset
hindi_data = pd.read_csv('hindi_hatespeech.tsv', sep='\t')
hindi_data = hindi_data.loc[:,"text":"task_1"]
hindi_data.dropna(subset = ["text"], inplace=True)
hindi_data.dropna(subset = ["task_1"], inplace=True)
hindi_data['task_1'] = hindi_data['task_1'].map({'HOF': 1, 'NOT': 0})
hindi_data['text'] = hindi_data['text'].str.replace('[{}]'.format(string.punctuation), ' ')
hindi_data['text'] = hindi_data['text'].str.replace('[{}]'.format('।'), '')
hindi_stop_word_list = pd.read_csv('stopword.txt', sep='\s+', header=None)
hindi_stop_word_list = hindi_stop_word_list[0].tolist()
hindi_data['text'] = hindi_data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in (hindi_stop_word_list)]))
hindi_training_data = hindi_data.sample(frac = 0.8)
hindi_test_data = hindi_data.drop(hindi_training_data.index)
# write a dataframe to tsv file
hindi_training_data.to_csv("hindi_training_data.tsv", sep="\t", index=False)
# write a dataframe to tsv file
hindi_test_data.to_csv("hindi_test_data.tsv", sep="\t", index=False)

In [ ]:
#Hindi Dataset Cleanup
def hindi_data_cleanup(hindi_data):
  cleaned_text = []
  for text in hindi_data:
      # remove multiple spaces
      text = re.sub(r' +', ' ', text)
      # remove newline
      text = re.sub(r'\n', ' ', text)
      cleaned_text.append(text)
  return cleaned_text

In [ ]:
#For Creating Train and Validation data
char_based = False
if char_based:
    tokenizer = lambda s: list(s) # char-based
else:
    tokenizer = lambda s: s.split() # word-based

Text = data.Field(preprocessing=hindi_data_cleanup, tokenize=tokenizer, batch_first=True, include_lengths=True, fix_length=100)
Label = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

fields = [('text', Text), ('labels', Label)]
train_data_hindi, test_data_hindi = data.TabularDataset.splits(
    path = "/content",
    train = "hindi_training_data.tsv",
    test = "hindi_test_data.tsv",
    format='tsv',
    fields=fields,
    skip_header=True
)

seed = 42
train_data_hindi, valid_data_hindi = train_data_hindi.split(split_ratio=0.8, random_state=random.seed(seed))

In [ ]:
def create_iterator(train_data, valid_data, test_data, batch_size, device):    
  train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data),
          batch_size = batch_size,
          sort_key = lambda x: len(x.text), 
          sort_within_batch = True,
          device = device)
  return train_iterator, valid_iterator, test_iterator

def accuracy(probs, target):
  predictions = probs.argmax(dim=1)
  corrects = (predictions == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text, text_lengths = text.to(device), text_lengths.to(device)
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.labels.squeeze())
        acc = accuracy(predictions, batch.labels)        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.labels)
            acc = accuracy(predictions, batch.labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def run_train(epochs, model, train_iterator, valid_iterator, optimizer, criterion, model_type):
  best_valid_loss = float('inf')
  training_loss_list = []
  valid_loss_list = []
  for epoch in range(epochs):

      # train the model
      train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

      # evaluate the model
      valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

      # save the best model
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'saved_weights'+'_'+model_type+'.pt')
      
      print(f'Epoch: {epoch+1} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
      print(f'Epoch: {epoch+1} | Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')


In [ ]:
# Hyperparameters
lr = 1e-4
batch_size = 50
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 100  # each sentence has until 100 words
dev_size = 0.8 # split percentage to train\validation data
seed = 1
num_classes = 2
num_hidden_nodes = 93
hidden_dim2 = 128
num_layers = 2  # LSTM layers
bi_directional = False 
num_epochs = 7
num_hidden_nodes = 93

In [ ]:


class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, n_layers,
                 bidirectional, dropout, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_index)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim1,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.fc1 = nn.Linear(hidden_dim1 * 2, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True) 

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        rel = self.relu(cat)
        dense1 = self.fc1(rel)
        drop = self.dropout(dense1)
        preds = self.fc2(drop)
        return preds

In [ ]:
# Main Function

if __name__ == "__main__":
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
  path = '/content'
  path_data = os.path.join(path, "data")
  # parameters
  model_type = "LSTM"
  data_type = "token" # or: "morph"

  char_based = True
  if char_based:
      tokenizer = lambda s: list(s) # char-based
  else:
      tokenizer = lambda s: s.split() # word-based

  Text.build_vocab(train_data_hindi)
  Label.build_vocab(train_data_hindi)
  vocab_size = len(Text.vocab)
  to_train = True
  pad_index = Text.vocab.stoi[Text.pad_token]


  train_iterator, valid_iterator, test_iterator = create_iterator(train_data_hindi, valid_data_hindi, test_data_hindi, batch_size, device)

  # loss function
  loss_func = nn.CrossEntropyLoss()
  loss_func = loss_func.to(device)
  lstm_model = LSTM(vocab_size, embedding_size, num_hidden_nodes, hidden_dim2 , num_classes, num_layers, bi_directional, dropout_keep_prob, pad_index)
  if torch.cuda.is_available():
    lstm_model.cuda()

  # optimization algorithm
  optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

  torch.backends.cudnn.enabled = False
  run_train(num_epochs, lstm_model, train_iterator, valid_iterator, optimizer, loss_func, model_type)

  test_loss, test_acc = evaluate(lstm_model, test_iterator, loss_func)
  print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Epoch: 1 | Train Loss: 0.694 | Train Acc: 50.53%
Epoch: 1 | Val. Loss: 0.689 |  Val. Acc: 56.17%
Epoch: 2 | Train Loss: 0.681 | Train Acc: 59.28%
Epoch: 2 | Val. Loss: 0.683 |  Val. Acc: 57.34%
Epoch: 3 | Train Loss: 0.666 | Train Acc: 62.83%
Epoch: 3 | Val. Loss: 0.670 |  Val. Acc: 58.81%
Epoch: 4 | Train Loss: 0.630 | Train Acc: 65.23%
Epoch: 4 | Val. Loss: 0.626 |  Val. Acc: 65.36%
Epoch: 5 | Train Loss: 0.542 | Train Acc: 72.41%
Epoch: 5 | Val. Loss: 0.571 |  Val. Acc: 69.25%
Epoch: 6 | Train Loss: 0.455 | Train Acc: 79.38%
Epoch: 6 | Val. Loss: 0.543 |  Val. Acc: 72.09%
Epoch: 7 | Train Loss: 0.384 | Train Acc: 83.49%
Epoch: 7 | Val. Loss: 0.540 |  Val. Acc: 74.90%
Test Loss: 0.516 | Test Acc: 77.72%


Bangla Dataset

In [ ]:
# Read Bangla Dataset
bengali_data = pd.read_csv('bengali_hatespeech.csv', sep=',')
bengali_data = bengali_data.loc[:,"sentence":"hate"]
bengali_data['sentence'] = bengali_data['sentence'].str.replace('[{}]'.format(string.punctuation), ' ')
bengali_data['sentence'] = bengali_data['sentence'].str.replace('[{}]'.format('।'), '')
bengali_stop_word_list = pd.read_csv('banglastopword.txt', sep='\s+', header=None)
bengali_stop_word_list = bengali_stop_word_list[0].tolist()
bengali_data['sentence'] = bengali_data['sentence'].apply(lambda x: ' '.join([item for item in x.split() if item not in (bengali_stop_word_list)]))
bengali_data = bengali_data[bengali_data.sentence != ""]
bengali_data.dropna(subset = ["sentence"], inplace=True)
bengali_data.dropna(subset = ["hate"], inplace=True)
bengali_training_data = bengali_data.sample(frac = 0.8)
bengali_test_data = bengali_data.drop(bengali_training_data.index)
# write a dataframe to tsv file
bengali_training_data.to_csv("bengali_training_data.tsv", sep="\t", index=False)
# write a dataframe to tsv file
bengali_test_data.to_csv("bengali_test_data.tsv", sep="\t", index=False)

In [ ]:
# This cell is for keeping the size of both dataset equal. If you want the dataset equal, don't run this cell.
# bengali_data = bengali_data.sample(n=len(hindi_data))

In [ ]:
#Bengali Dataset Cleanup
def bengali_data_cleanup(bengali_data):
  cleaned_text = []
  for text in bengali_data:
      # remove multiple spaces
      text = re.sub(r' +', ' ', text)
      # remove newline
      text = re.sub(r'\n', ' ', text)
      cleaned_text.append(text)
  return cleaned_text

In [ ]:
Text = data.Field(preprocessing=bengali_data_cleanup, tokenize=tokenizer, batch_first=True, include_lengths=True, fix_length=100)
Label = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

fields = [('text', Text), ('labels', Label)]
train_data_bengali, test_data_bengali = data.TabularDataset.splits(
    path = "/content",
    train = "bengali_training_data.tsv",
    test = "bengali_test_data.tsv",
    format='tsv',
    fields=fields,
    skip_header=True
)

seed = 42
train_data_bengali, valid_data_bengali = train_data_bengali.split(split_ratio=0.8, random_state=random.seed(seed))

In [ ]:
# Main Function
if __name__ == "__main__":
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
  path = '/content'
  path_data = os.path.join(path, "data")
  # parameters
  model_type = "LSTM"
  data_type = "token" 

  char_based = True
  if char_based:
      tokenizer = lambda s: list(s) # char-based
  else:
      tokenizer = lambda s: s.split() # word-based

  Text.build_vocab(train_data_bengali)
  Label.build_vocab(train_data_bengali)
  vocab_size = len(Text.vocab)
  pad_index = Text.vocab.stoi[Text.pad_token]


  train_iterator, valid_iterator, test_iterator = create_iterator(train_data_bengali, valid_data_bengali, test_data_bengali, batch_size, device)

  # loss function
  loss_func = nn.CrossEntropyLoss()
  loss_func = loss_func.to(device)
  lstm_model = LSTM(vocab_size, embedding_size, num_hidden_nodes, hidden_dim2 , num_classes, num_layers, bi_directional, dropout_keep_prob, pad_index)
  if torch.cuda.is_available():
    lstm_model.cuda()

  # optimization algorithm
  optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)
  # train and evaluation

  torch.backends.cudnn.enabled = False
  run_train(num_epochs, lstm_model, train_iterator, valid_iterator, optimizer, loss_func, model_type)

  # load weights
  lstm_model.load_state_dict(torch.load(os.path.join(path, "saved_weights_LSTM.pt")))
  # predict
  test_loss, test_acc = evaluate(lstm_model, test_iterator, loss_func)
  print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Epoch: 1 | Train Loss: 0.616 | Train Acc: 67.74%
Epoch: 1 | Val. Loss: 0.544 |  Val. Acc: 73.36%
Epoch: 2 | Train Loss: 0.482 | Train Acc: 78.41%
Epoch: 2 | Val. Loss: 0.512 |  Val. Acc: 75.43%
Epoch: 3 | Train Loss: 0.417 | Train Acc: 82.93%
Epoch: 3 | Val. Loss: 0.408 |  Val. Acc: 83.55%
Epoch: 4 | Train Loss: 0.401 | Train Acc: 84.44%
Epoch: 4 | Val. Loss: 0.399 |  Val. Acc: 84.69%
Epoch: 5 | Train Loss: 0.450 | Train Acc: 80.35%
Epoch: 5 | Val. Loss: 0.486 |  Val. Acc: 79.29%
Epoch: 6 | Train Loss: 0.424 | Train Acc: 82.63%
Epoch: 6 | Val. Loss: 0.403 |  Val. Acc: 84.27%
Epoch: 7 | Train Loss: 0.371 | Train Acc: 85.51%
Epoch: 7 | Val. Loss: 0.382 |  Val. Acc: 85.03%
Test Loss: 0.371 | Test Acc: 85.29%
